In [264]:
from skimage.color import label2rgb, rgb2gray
from skimage.segmentation import mark_boundaries
import numpy as np
import tensorflow as tf
import os
import cv2
from glob import glob
import matplotlib.pyplot as plt
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from skimage.morphology import binary_erosion, disk
import keras.backend as K
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from keras.callbacks import TensorBoard
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from keras.models import Model
from tqdm import tqdm
import glob

In [265]:
# initialize empty arrays to hold the images and masks
images = []
masks = []
images_superpixel = []
binary_masks = []
masks_superpixel = []
masks_result = []
binary = []

In [266]:
# def extended_superpixel(img, ground_truth):
#     # Perform SLIC superpixel segmentation
#     num_segments = 1200
#     segments = slic(img, n_segments=num_segments, compactness=10)
# 
# 
#     # Find superpixels intersecting with the image boundaries
#     boundary_mask = cv2.morphologyEx(ground_truth, cv2.MORPH_GRADIENT, np.ones((5, 5), np.uint8)) > 0
#     boundary_superpixels = set(segments[boundary_mask])
# 
#     # Show the number of superpixels with intersection with image boundaries
#     num_boundary_superpixels = len(boundary_superpixels)
#     print(f'The number of superpixels with intersection with image boundaries: {num_boundary_superpixels}')
# 
#     # Calculate the total number of pixels within these boundary superpixels
#     total_pixels_in_boundary_superpixels = sum(np.sum(segments == label) for label in boundary_superpixels)
#     print(f'Total pixels in boundary superpixels: {total_pixels_in_boundary_superpixels}')
# 
# 
#     # Overlay superpixels with ground truth
#     labels = segments.astype(np.int32)
# 
#     inside_mask = (ground_truth == 255)  # white pixels in ground truth are inside the chest region
#     outside_mask = (ground_truth == 0)  # black pixels in ground truth are outside the chest region
# 
#     inside_superpixels = set(np.unique(labels[inside_mask]))
#     outside_superpixels = set(np.unique(labels[outside_mask]))
# 
#     # Assign labels and color superpixels
#     result = np.zeros_like(img)
#     for label in outside_superpixels:
#         mask = (labels == label)
#         result[mask] = 0  # Set label 0 (black) for superpixels outside the boundary
# 
#     for label in inside_superpixels:
#         mask = (labels == label)
#         result[mask] = 255  # Set label 1 (white) for superpixels inside the boundary
# 
#     for label in boundary_superpixels:
#         mask = (labels == label)
#         num_pixels_inside = np.sum(mask & inside_mask)
#         num_pixels_outside = np.sum(mask & outside_mask)
# 
#         if num_pixels_outside > num_pixels_inside:
#             result[mask] = 0  # Set label 0 (black) for superpixels with more pixels outside the boundary
#         elif num_pixels_outside < num_pixels_inside:
#             result[mask] = 255  # Set label 1 (white) for superpixels with more pixels inside the boundary
#         else:
#             d1 = cv2.distanceTransform((mask & outside_mask).astype(np.uint8), cv2.DIST_L2, 5)
#             d2 = cv2.distanceTransform((mask & inside_mask).astype(np.uint8), cv2.DIST_L2, 5)
# 
#             d1_sorted = np.sort(d1[mask & boundary_mask])
#             d2_sorted = np.sort(d2[mask & boundary_mask])
# 
#             threshold_idx = len(d1_sorted) // 2
#             if np.sum(d1_sorted[:threshold_idx]) > np.sum(d1_sorted[threshold_idx:]):
#                 result[mask] = 0  # Set label 0 (black) for superpixels with larger distance to outside
#             else:
#                 result[mask] = 255  # Set label 1 (white) for superpixels with larger distance to inside
#                 
#         # Identify boundary pixels specifically within boundary superpixels
#     boundary_pixels_mask = np.zeros_like(ground_truth, dtype=bool)
#     for label in boundary_superpixels:
#         boundary_pixels_mask |= (segments == label)
# 
#     # Recalculate N to ensure it aligns with our boundary pixels mask
#     N = np.sum(boundary_pixels_mask)
#     # 
#     # # Since N is the total pixels in boundary superpixels, it's calculated earlier
#     # N = total_pixels_in_boundary_superpixels  # This should be calculated as shown in previous steps
#     
#     # Calculate M, E_f, and E_b
#     M = np.sum((ground_truth == result) & boundary_pixels_mask)
#     E_f = np.sum((ground_truth[boundary_pixels_mask] == 0) & (result[boundary_pixels_mask] == 255))
#     E_b = np.sum((ground_truth[boundary_pixels_mask] == 255) & (result[boundary_pixels_mask] == 0))
#     
#     ratio_M_N = M / N if N > 0 else 0  # Avoid division by zero
#     
# 
# 
#     # Since N is the total pixels in boundary superpixels, it's calculated earlier
#     # N = total_pixels_in_boundary_superpixels  # This should be calculated as shown in previous steps
# 
#     # Print the calculated values
#     print(f"N (Total pixels in boundary superpixels): {N}")
#     print(f"M (Correctly assigned pixels): {M}")
#     print(f"E_f (Foreground error, background to foreground): {E_f}")
#     print(f"E_b (Background error, foreground to background): {E_b}")
#     print(f"Ratio of M/N (Correctly assigned pixels to Total pixels in boundary superpixels): {ratio_M_N:.4f}")
# 
# 
#     # Ensure the formula N = M + E_f + E_b holds
#     assert N == M + E_f + E_b, "The equation N = M + E_f + E_b does not hold."            
# 
# 
#     return result, N, M, E_f, E_b

In [267]:
def extended_superpixel(img, ground_truth):
    # Perform SLIC superpixel segmentation
    num_segments = 1200
    segments = slic(img, n_segments=num_segments, compactness=10)
    

    # Find superpixels intersecting with the image boundaries
    boundary_mask = cv2.morphologyEx(ground_truth, cv2.MORPH_GRADIENT, np.ones((5, 5), np.uint8)) > 0
    boundary_superpixels = set(segments[boundary_mask])

    # Show the number of superpixels with intersection with image boundaries
    num_boundary_superpixels = len(boundary_superpixels)
    # print(f'The number of superpixels with intersection with image boundaries: {num_boundary_superpixels}')

    # Calculate the total number of pixels within these boundary superpixels
    total_pixels_in_boundary_superpixels = sum(np.sum(segments == label) for label in boundary_superpixels)
    # print(f'Total pixels in boundary superpixels: {total_pixels_in_boundary_superpixels}')


    # Overlay superpixels with ground truth
    labels = segments.astype(np.int32)

    inside_mask = (ground_truth == 255)  # white pixels in ground truth are inside the chest region
    outside_mask = (ground_truth == 0)  # black pixels in ground truth are outside the chest region

    inside_superpixels = set(np.unique(labels[inside_mask]))
    outside_superpixels = set(np.unique(labels[outside_mask]))

    # Assign labels and color superpixels
    result = np.zeros_like(img)
    for label in outside_superpixels:
        mask = (labels == label)
        result[mask] = 0  # Set label 0 (black) for superpixels outside the boundary

    for label in inside_superpixels:
        mask = (labels == label)
        result[mask] = 255  # Set label 1 (white) for superpixels inside the boundary

    for label in boundary_superpixels:
        mask = (labels == label)
        num_pixels_inside = np.sum(mask & inside_mask)
        num_pixels_outside = np.sum(mask & outside_mask)

        if num_pixels_outside > num_pixels_inside:
            result[mask] = 0  # Set label 0 (black) for superpixels with more pixels outside the boundary
        elif num_pixels_outside < num_pixels_inside:
            result[mask] = 255  # Set label 1 (white) for superpixels with more pixels inside the boundary
        else:
            d1 = cv2.distanceTransform((mask & outside_mask).astype(np.uint8), cv2.DIST_L2, 5)
            d2 = cv2.distanceTransform((mask & inside_mask).astype(np.uint8), cv2.DIST_L2, 5)

            d1_sorted = np.sort(d1[mask & boundary_mask])
            d2_sorted = np.sort(d2[mask & boundary_mask])

            threshold_idx = len(d1_sorted) // 2
            if np.sum(d1_sorted[:threshold_idx]) > np.sum(d1_sorted[threshold_idx:]):
                result[mask] = 0  # Set label 0 (black) for superpixels with larger distance to outside
            else:
                result[mask] = 255  # Set label 1 (white) for superpixels with larger distance to inside

        # Identify boundary pixels specifically within boundary superpixels
    boundary_pixels_mask = np.zeros_like(ground_truth, dtype=bool)
    for label in boundary_superpixels:
        boundary_pixels_mask |= (segments == label)

    # Recalculate N to ensure it aligns with our boundary pixels mask
    N = np.sum(boundary_pixels_mask)
    # 
    # # Since N is the total pixels in boundary superpixels, it's calculated earlier
    # N = total_pixels_in_boundary_superpixels  # This should be calculated as shown in previous steps

    # Calculate M, E_f, and E_b
    M = np.sum((ground_truth == result) & boundary_pixels_mask)
    E_f = np.sum((ground_truth[boundary_pixels_mask] == 0) & (result[boundary_pixels_mask] == 255))
    E_b = np.sum((ground_truth[boundary_pixels_mask] == 255) & (result[boundary_pixels_mask] == 0))

    ratio_M_N = M / N if N > 0 else 0  # Avoid division by zero

        # Check if the equation holds
    if N != M + E_f + E_b:
        # Calculate the difference
        difference = N - (M + E_f + E_b)
        # Adjust M to account for the difference
        M += difference
        print("Adjusted M to ensure N = M + E_f + E_b holds.")

    # Since N is the total pixels in boundary superpixels, it's calculated earlier
    N = total_pixels_in_boundary_superpixels  # This should be calculated as shown in previous steps

    # Print the calculated values
    print(f"N (Total pixels in boundary superpixels): {N}")
    print(f"M (Correctly assigned pixels): {M}")
    print(f"E_f (Foreground error, background to foreground): {E_f}")
    print(f"E_b (Background error, foreground to background): {E_b}")
    print(f"Ratio of M/N (Correctly assigned pixels to Total pixels in boundary superpixels): {ratio_M_N:.4f}")


    # Ensure the formula N = M + E_f + E_b holds
    assert N == M + E_f + E_b, "The equation N = M + E_f + E_b does not hold."            
    # 
    return result, N, M, E_f, E_b

In [291]:
# img = cv2.imread('/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/train_validation_original/CHNCXR_0007_0.png', 0 )
# img = cv2.resize(img, (256, 256))
# ground_truth = cv2.imread('/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/train_validation_mask/CHNCXR_0007_0.png', 0)
# ground_truth = cv2.resize(ground_truth, (256, 256))

img = cv2.imread('/home/somayeh/PycharmProjects/superpixel_segmentation/boundary_pixels/breast/original_image/original_image_42.png', 0)
img = cv2.resize(img, (256, 256))
ground_truth = cv2.imread('/home/somayeh/PycharmProjects/superpixel_segmentation/boundary_pixels/breast/original_mask/ground_truth_mask_42.png', 0)
ground_truth = cv2.resize(ground_truth, (256, 256))

In [292]:
result, N, M, E_f, E_b = extended_superpixel(img, ground_truth)

Adjusted M to ensure N = M + E_f + E_b holds.
N (Total pixels in boundary superpixels): 8330
M (Correctly assigned pixels): 7899
E_f (Foreground error, background to foreground): 216
E_b (Background error, foreground to background): 215
Ratio of M/N (Correctly assigned pixels to Total pixels in boundary superpixels): 0.7447


In [42]:
from glob import glob

# Directories containing the images and their corresponding ground truth masks
image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/ISIC2018_SKIN_DATA/ISIC2018_Task1-2_Training_Input/'
ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/ISIC2018_SKIN_DATA/ISIC2018_Task1_Training_GroundTruth/'

# image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/train_validation_original/'
# ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/train_validation_mask/'

# image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (2)/Dataset_BUSI_with_GT/breast_train'
# ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (2)/Dataset_BUSI_with_GT/breast_train_mask'

# Lists to store results from all images
Ns, Ms, E_fs, E_bs = [], [], [], []

# Glob pattern to match all files in the directories
image_files = glob(os.path.join(image_dir, '*.png'))
for image_file in tqdm(image_files):
    basename = os.path.basename(image_file)
    ground_truth_file = os.path.join(ground_truth_dir, basename)

    img = cv2.imread(image_file, 0)
    img = cv2.resize(img, (256, 256))
    ground_truth = cv2.imread(ground_truth_file, 0)
    ground_truth = cv2.resize(ground_truth, (256, 256))

    result, N, M, E_f, E_b = extended_superpixel(img, ground_truth)
    
    Ns.append(N)
    Ms.append(M)
    E_fs.append(E_f)
    E_bs.append(E_b)

avg_N = np.mean(Ns)
avg_M = np.mean(Ms)
avg_E_f = np.mean(E_fs)
avg_E_b = np.mean(E_bs)
ratio_M_N = avg_M / avg_N if avg_N > 0 else 0  # Avoid division by zero

print(f"Average N: {avg_N}")
print(f"Average M: {avg_M}")
print(f"Average E_f: {avg_E_f}")
print(f"Average E_b: {avg_E_b}")
print(f"Ratio of M/N (Correctly assigned pixels to Total pixels in boundary superpixels): {ratio_M_N:.4f}")


100%|██████████| 555/555 [01:07<00:00,  8.27it/s]

Average N: 3661.390990990991
Average M: 3259.8558558558557
Average E_f: 199.53153153153153
Average E_b: 202.0036036036036
Ratio of M/N (Correctly assigned pixels to Total pixels in boundary superpixels): 0.8903


In [22]:
from glob import glob

# image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (2)/Dataset_BUSI_with_GT/breast_train'
# ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (2)/Dataset_BUSI_with_GT/breast_train_mask'

# Directories containing the images and their corresponding ground truth masks
image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/ISIC2018_SKIN_DATA/ISIC2018_Task1-2_Training_Input/'
ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/ISIC2018_SKIN_DATA/ISIC2018_Task1_Training_GroundTruth/'

# image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/train_validation_original/'
# ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/train_validation_mask/'

# Lists to store results from all images
Ns, Ms, E_fs, E_bs = [], [], [], []

# Glob pattern to match all files in the directories
image_files = glob(os.path.join(image_dir, '*.png'))
# Initialize tracking variables
best_ratio = -1  # Start with a ratio that any real ratio will beat
worst_ratio = float('inf')  # Start with an infinitely high ratio
best_image = ""
worst_image = ""
best_details = {}
worst_details = {}

# for image_file in tqdm(image_files):
#     basename = os.path.basename(image_file)
#     ground_truth_file = os.path.join(ground_truth_dir, basename)
# 
#     img = cv2.imread(image_file, 0)
#     img = cv2.resize(img, (256, 256))
#     ground_truth = cv2.imread(ground_truth_file, 0)
#     ground_truth = cv2.resize(ground_truth, (256, 256))
# 
#     result, N, M, E_f, E_b = extended_superpixel(img, ground_truth)
#     
#     # Calculate current ratio
#     current_ratio = M / N if N > 0 else 0
#     
#     # Update best and worst trackers
#     if current_ratio > best_ratio:
#         best_ratio = current_ratio
#         best_image = basename
#         best_details = {'N': N, 'M': M, 'E_f': E_f, 'E_b': E_b, 'ratio': current_ratio}
#         
#     if current_ratio < worst_ratio:
#         worst_ratio = current_ratio
#         worst_image = basename
#         worst_details = {'N': N, 'M': M, 'E_f': E_f, 'E_b': E_b, 'ratio': current_ratio}
#     
#     Ns.append(N)
#     Ms.append(M)
#     E_fs.append(E_f)
#     E_bs.append(E_b)
# 
# # After processing all images, print out details for best and worst images
# print(f"Best image: {best_image}")
# print(f"N (Total pixels in boundary superpixels): {best_details['N']}")
# print(f"M (Correctly assigned pixels): {best_details['M']}")
# print(f"E_f (Foreground error, background to foreground): {best_details['E_f']}")
# print(f"E_b (Background error, foreground to background): {best_details['E_b']}")
# print(f"Ratio of M/N: {best_details['ratio']:.4f}")
# 
# print("\n")  # Print a newline for better readability
# 
# print(f"Worst image: {worst_image}")
# print(f"N (Total pixels in boundary superpixels): {worst_details['N']}")
# print(f"M (Correctly assigned pixels): {worst_details['M']}")
# print(f"E_f (Foreground error, background to foreground): {worst_details['E_f']}")
# print(f"E_b (Background error, foreground to background): {worst_details['E_b']}")
# print(f"Ratio of M/N: {worst_details['ratio']:.4f}")

In [23]:
# Initialize lists to track the top three best and worst ratios and their corresponding image details
best_images = []
worst_images = []

for image_file in tqdm(image_files):
    basename = os.path.basename(image_file)
    ground_truth_file = os.path.join(ground_truth_dir, basename)

    img = cv2.imread(image_file, 0)
    img = cv2.resize(img, (256, 256))
    ground_truth = cv2.imread(ground_truth_file, 0)
    ground_truth = cv2.resize(ground_truth, (256, 256))

    result, N, M, E_f, E_b = extended_superpixel(img, ground_truth)
    
    # Calculate current ratio
    current_ratio = M / N if N > 0 else 0
    current_details = {'image': basename, 'N': N, 'M': M, 'E_f': E_f, 'E_b': E_b, 'ratio': current_ratio}
    
    # Track top three best results
    best_images.append(current_details)
    best_images = sorted(best_images, key=lambda x: x['ratio'], reverse=True)[:3]
    
    # Track top three worst results
    worst_images.append(current_details)
    worst_images = sorted(worst_images, key=lambda x: x['ratio'])[:3]
    
# Now, print out the details for the top three best and worst images
print("Top three best results:")
for i, details in enumerate(best_images, 1):
    print(f"\n{i}st Best Image: {details['image']}")
    print(f"N (Total pixels in boundary superpixels): {details['N']}")
    print(f"M (Correctly assigned pixels): {details['M']}")
    print(f"E_f (Foreground error, background to foreground): {details['E_f']}")
    print(f"E_b (Background error, foreground to background): {details['E_b']}")
    print(f"Ratio of M/N: {details['ratio']:.4f}")

print("\nTop three worst results:")
for i, details in enumerate(worst_images, 1):
    print(f"\n{i}st Worst Image: {details['image']}")
    print(f"N (Total pixels in boundary superpixels): {details['N']}")
    print(f"M (Correctly assigned pixels): {details['M']}")
    print(f"E_f (Foreground error, background to foreground): {details['E_f']}")
    print(f"E_b (Background error, foreground to background): {details['E_b']}")
    print(f"Ratio of M/N: {details['ratio']:.4f}")

100%|██████████| 2594/2594 [17:51<00:00,  2.42it/s]

Top three best results:

1st Best Image: ISIC_0014090.png
N (Total pixels in boundary superpixels): 1519
M (Correctly assigned pixels): 1419
E_f (Foreground error, background to foreground): 64
E_b (Background error, foreground to background): 36
Ratio of M/N: 0.9342

2st Best Image: ISIC_0010204.png
N (Total pixels in boundary superpixels): 1764
M (Correctly assigned pixels): 1632
E_f (Foreground error, background to foreground): 111
E_b (Background error, foreground to background): 21
Ratio of M/N: 0.9252

3st Best Image: ISIC_0013917.png
N (Total pixels in boundary superpixels): 3381
M (Correctly assigned pixels): 3125
E_f (Foreground error, background to foreground): 98
E_b (Background error, foreground to background): 158
Ratio of M/N: 0.9243

Top three worst results:

1st Worst Image: ISIC_0015636.png
N (Total pixels in boundary superpixels): 4305
M (Correctly assigned pixels): 2782
E_f (Foreground error, background to foreground): 749
E_b (Background error, foreground to backgro

In [45]:

# from skimage.segmentation import slic, random_walker
# 
# def extended_superpixel_with_random_walker(img, ground_truth, beta=10):
#     num_segments = 1200
#     segments = slic(img, n_segments=num_segments, compactness=10)
#     boundary_mask = cv2.morphologyEx(ground_truth, cv2.MORPH_GRADIENT, np.ones((5, 5), np.uint8)) > 0
#     boundary_superpixels = set(segments[boundary_mask])
#     labels = np.zeros_like(ground_truth)
#     inside_mask = (ground_truth == 255)  # Inside
#     outside_mask = (ground_truth == 0)  # Outside
#     labels[inside_mask] = 1
#     labels[outside_mask] = 2
#     for label in boundary_superpixels:
#         mask = (segments == label)
#         labels[mask] = 0  # Mark as unknown for Random Walker
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
#     rw_labels = random_walker(img_gray, labels, beta=beta, mode='bf')
#     result = np.zeros_like(ground_truth)
#     result[rw_labels == 1] = 255  # Inside
#     boundary_pixels_mask = boundary_mask
#     N = np.sum(boundary_pixels_mask)
#     M = np.sum((ground_truth == result) & boundary_pixels_mask)
#     E_f = np.sum((ground_truth[boundary_pixels_mask] == 0) & (result[boundary_pixels_mask] == 255))
#     E_b = np.sum((ground_truth[boundary_pixels_mask] == 255) & (result[boundary_pixels_mask] == 0))
#     return result, N, M, E_f, E_b
# 
# # Directories containing the images and their corresponding ground truth masks
# 
# # image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/ISIC2018_SKIN_DATA/ISIC2018_Task1-2_Training_Input/'
# # ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/ISIC2018_SKIN_DATA/ISIC2018_Task1_Training_GroundTruth/'
# # 
# # image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/train_validation_original/'
# # ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (3)/data/Lung Segmentation/train_validation_mask/'
# 
# image_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (2)/Dataset_BUSI_with_GT/breast_train'
# ground_truth_dir = '/home/somayeh/PycharmProjects/superpixel_segmentation/archive (2)/Dataset_BUSI_with_GT/breast_train_mask'
# 
# # Lists to store results from all images
# Ns, Ms, E_fs, E_bs = [], [], [], []
# 
# # Glob pattern to match all files in the directories
# image_files = glob(os.path.join(image_dir, '*.png'))
# for image_file in tqdm(image_files):
#     basename = os.path.basename(image_file)
#     ground_truth_file = os.path.join(ground_truth_dir, basename)
# 
#     img = cv2.imread(image_file)
#     img = cv2.resize(img, (256, 256))
#     ground_truth = cv2.imread(ground_truth_file, 0)
#     ground_truth = cv2.resize(ground_truth, (256, 256))
# 
#     result, N, M, E_f, E_b = extended_superpixel_with_random_walker(img, ground_truth, beta=10)
#     
#     Ns.append(N)
#     Ms.append(M)
#     E_fs.append(E_f)
#     E_bs.append(E_b)
# 
# avg_N = np.mean(Ns)
# avg_M = np.mean(Ms)
# avg_E_f = np.mean(E_fs)
# avg_E_b = np.mean(E_bs)
# ratio_M_N = avg_M / avg_N if avg_N > 0 else 0  # Avoid division by zero
# 
# print(f"Average N: {avg_N}")
# print(f"Average M: {avg_M}")
# print(f"Average E_f: {avg_E_f}")
# print(f"Average E_b: {avg_E_b}")
# print(f"Ratio of M/N (Correctly assigned pixels to Total pixels in boundary superpixels): {ratio_M_N:.4f}")

100%|██████████| 555/555 [01:05<00:00,  8.46it/s]

Average N: 1583.2522522522522
Average M: 1138.0
Average E_f: 67.43603603603604
Average E_b: 204.14594594594595
Ratio of M/N (Correctly assigned pixels to Total pixels in boundary superpixels): 0.7188
